## Lab. 12

### Solve the following problem using Genetic Algorithms:


Problem: Weighted N-Queen Problem


You are given an N×N chessboard, and each cell of the board has an associated weight. Your task is to find a valid placement of N queens such that the total weight of the queens is maximized, and no two queens threaten each other.





In the traditional N-Queen Problem, the goal is to place N queens on an N×N chessboard in such a way that no two queens threaten each other. In this variation, we introduce weights to the queens and aim to find a placement that maximizes the total weight of the queens while satisfying the constraint of non-threatening positions.


Constraints:

1. There should be exactly one queen in each row and each column.
2. No two queens should be placed in the same diagonal, i.e., they should not threaten each other.
3. The placement should maximize the total weight of the queens.


Representation:

Use a permutation-based representation. Each permutation represents the column position of the queen for each row. 

For example, if N=4, a valid permutation [2, 4, 1, 3] indicates that the queen in the first row is placed in column 2, the queen in the second row is placed in column 4, and so on.


Genetic Algorithm Steps:

1. *Initialization*: Generate an initial population of permutations randomly.

2. *Fitness Evaluation*: Evaluate the fitness of each permutation by calculating the total weight of the queens while considering the non-threatening positions.

3. *Selection*: Select a subset of permutations from the population based on their fitness, using selection techniques like tournament selection or roulette wheel selection.

4. *Crossover*: Perform crossover (recombination) on the selected permutations to create new offspring permutations.

5. *Mutation*: Introduce random changes (mutations) in the offspring permutations to maintain diversity in the population.

6. *Fitness Evaluation for the new individuals*: Evaluate the fitness of the new population.

7. *Form the new population*: Select the surviving individuals based on scores, with chances direct proportional with their performance.

8. Repeat steps 3-7 for a certain number of generations or until a termination condition is met (e.g., a maximum number of iterations or a satisfactory solution is found).


9. *Termination*: Return the best-performing individual (permutation) found as the solution to the problem.

Note: The fitness function used in this problem should calculate the total weight of the queens based on the positions specified by the permutation. Additionally, the fitness function should penalize solutions that violate the non-threatening constraint by assigning a lower fitness score to such permutations.

In [129]:
from random import shuffle
from random import random
from random import randint
from random import randrange
from copy import deepcopy

N = 8
chessboard =  [
    [3, 4, 1, 5, 0, 4, 7, 3],
    [7, 2, 3, 1, 1, 2, 2, 4],
    [4, 2, 8, 2, 1, 3, 2, 6],
    [9, 1, 2, 3, 4, 5, 1, 4],
    [2, 2, 1, 4, 6, 3, 9, 3],
    [1, 5, 5, 2, 3, 1, 5, 1],
    [4, 7, 1, 3, 1, 6, 3, 6],
    [9, 7, 1, 3, 3, 6, 8, 7]
]

BASE_PERMUTATION = [i for i in range(N)]
CROSSOVER_PIVOTS = (3, 5)
POPULATION_SIZE = 10
BASE_FITNESS = 1

def generateIndividual():
    individual = deepcopy(BASE_PERMUTATION)
    shuffle(individual)
    return individual

def generatePopulation():
    return [generateIndividual() for i in range(POPULATION_SIZE)]

def isSolution(individual):
    for i in range(N):
        for j in range(i + 1, N):
            if abs(i - j) == abs(individual[i] - individual[j]):
                return False
    return True

def fitnessOfIndividual(individual):
    if not isSolution(individual):
        return BASE_FITNESS
    fitness_of_individual = BASE_FITNESS + sum(chessboard[i][individual[i]] for i in range(N)) # i is the row and individual[i] is the column
    return fitness_of_individual

def fitnessOfEachIndividual(population):
    return [fitnessOfIndividual(individual) for individual in population]

def selectParents(population):
    # roulette wheel selection
    # select two parents from the population based on the fitness - 
    # the better the fitness, the higher the chance to be selected
    intervals_of_each_individual = [0]
    for individual in population:
        intervals_of_each_individual.append(intervals_of_each_individual[len(intervals_of_each_individual) - 1] + fitnessOfIndividual(individual))
    #print(intervals)
    number_for_first_parent = randrange(intervals_of_each_individual[len(intervals_of_each_individual) - 1])
    number_for_second_parent = randrange(intervals_of_each_individual[len(intervals_of_each_individual) - 1])
    first_parent, second_parent = None, None
    for i in range(1, len(intervals_of_each_individual)):
        if number_for_first_parent < intervals_of_each_individual[i] and first_parent is None:
            first_parent = population[i-1] 
        if number_for_second_parent < intervals_of_each_individual[i] and second_parent is None:
            second_parent = population[i-1]
    return (first_parent, second_parent) # btw the parents can be the same entity, this is suboptimal

def crossover(parents):
    # Order crossover (OX1) https://en.wikipedia.org/wiki/Crossover_%28genetic_algorithm%29#Crossover_for_permutations
    # This function uses only two crossover pivots (they are defined in the CROSSOVER_PIVOTS constant)
    # This could benefit from being more random (for example, taking random pivots)
    UNINITIALIZED_GENE = -1
    child = [UNINITIALIZED_GENE] * N 
    for i in range(CROSSOVER_PIVOTS[0], CROSSOVER_PIVOTS[1] + 1):
        child[i] = parents[0][i]

    i = 0
    for gene in parents[1]:
        if gene not in child:
            while (i < N and child[i] != UNINITIALIZED_GENE):
                i += 1
            #print(str(gene) + ' ' + str(i))
            #print(child)
            child[i] = gene
    return child


def mutate(individual, probability_of_mutation=0.1): # probability_of_mutation must be between 0 and 1
    if probability_of_mutation > random():
        random_queen_index = randint(0, N-1)
        another_random_queen_index = randint(0, N-1)
        while random_queen_index == another_random_queen_index:
            another_random_queen_index = randint(0, N-1)
        individual[random_queen_index], individual[another_random_queen_index] = individual[another_random_queen_index], individual[random_queen_index]
    return individual

In [163]:
def genetic_algorithm(populationReplacementStrategy):
    # complete genetic algorithm
    current_population = generatePopulation()
    best_population = current_population

    for i in range(NUMBER_OF_GENERATIONS):
        current_fitness_scores = fitnessOfEachIndividual(current_population)
        # print(f'Total fitness of era {i}: {sum(current_fitness_scores)}')

        new_population = []

        for _ in range(POPULATION_SIZE):
            parents = selectParents(current_population)
            offspring = crossover(parents)
            mutated_offspring = mutate(offspring, MUTATION_RATE) 

            offspring_fitness = fitnessOfIndividual(mutated_offspring)
            first_parent_fitness = fitnessOfIndividual(parents[0])
            second_parent_fitness = fitnessOfIndividual(parents[1])
            if first_parent_fitness > offspring_fitness and first_parent_fitness > second_parent_fitness:
                new_population.append(parents[0])
            elif second_parent_fitness > offspring_fitness and second_parent_fitness > first_parent_fitness:
                new_population.append(parents[1])
            else:
                new_population.append(mutated_offspring)

        
        if sum(current_fitness_scores) == BASE_FITNESS * POPULATION_SIZE: # just replace it if there are no fit members
            current_population = new_population 
        else:
            current_population = populationReplacementStrategy(current_population, new_population)

        if sum(fitnessOfEachIndividual(current_population)) >= sum(fitnessOfEachIndividual(best_population)):
            best_population = current_population

    return best_population


# Here are the 4 population replacement strategies:
def acceptNewPopulation(current_population, new_population):
    return new_population

def acceptNewPopulationIfItHasAtLeastOneFitMember(current_population, new_population):
    if sum(fitnessOfEachIndividual(new_population)) > BASE_FITNESS * POPULATION_SIZE: 
        return new_population 
    return current_population

def acceptNewPopulationIfItIsBetterThanTheCurrentOne(current_population, new_population):
    if sum(fitnessOfEachIndividual(new_population)) > sum(fitnessOfEachIndividual(current_population)): 
            return new_population 
    return current_population

# this one might be highly dependant on how EXTRA_FITNESS_SCORE_NEEDED is chosen
def acceptNewPopulationIfItIsMuchBetterThanTheCurrentOne(current_population, new_population):
    EXTRA_FITNESS_SCORE_NEEDED = 150
    if sum(fitnessOfEachIndividual(new_population)) > sum(fitnessOfEachIndividual(current_population)) + EXTRA_FITNESS_SCORE_NEEDED: 
         return new_population 
    return current_population
    
# Test the complete genetic algorithm
MUTATION_RATE = 0.9
NUMBER_OF_GENERATIONS = 100
final_population = genetic_algorithm(acceptNewPopulationIfItIsMuchBetterThanTheCurrentOne)
print(final_population)

[[6, 3, 1, 7, 5, 0, 2, 4], [6, 3, 1, 7, 5, 0, 2, 4], [6, 3, 1, 7, 5, 0, 2, 4], [7, 3, 1, 6, 5, 0, 2, 4], [6, 3, 1, 7, 5, 0, 2, 4], [6, 3, 5, 7, 1, 0, 2, 4], [6, 3, 1, 7, 5, 0, 2, 4], [6, 3, 1, 7, 5, 0, 2, 4], [6, 3, 1, 7, 5, 0, 2, 4], [6, 3, 1, 7, 5, 0, 2, 4]]


In [119]:
best_individual = max(final_population, key=lambda individual: fitnessOfIndividual(individual))

print(f'The best individual is {best_individual} and their value is {fitnessOfIndividual(best_individual)}')


The best individual is [3, 7, 0, 2, 5, 1, 6, 4] and their value is 30


In [166]:
from statistics import mean

def studyPopulationReplacementStrategies():
      l1 = []
      l2 = []
      l3 = []
      l4 = []
      epochs = 100
      for i in range (epochs):
            l1.append(max(genetic_algorithm(acceptNewPopulation), key=lambda individual: fitnessOfIndividual(individual)))
            l2.append(max(genetic_algorithm(acceptNewPopulationIfItHasAtLeastOneFitMember), key=lambda individual: fitnessOfIndividual(individual)))
            l3.append(max(genetic_algorithm(acceptNewPopulationIfItIsBetterThanTheCurrentOne), key=lambda individual: fitnessOfIndividual(individual)))
            l4.append(max(genetic_algorithm(acceptNewPopulationIfItIsMuchBetterThanTheCurrentOne), key=lambda individual: fitnessOfIndividual(individual)))

      result1 = mean([fitnessOfIndividual(individual) for individual in l1])
      result2 = mean([fitnessOfIndividual(individual) for individual in l2])
      result3 = mean([fitnessOfIndividual(individual) for individual in l3])
      result4 = mean([fitnessOfIndividual(individual) for individual in l4])

      print(f'Genetic algorithm with acceptNewPopulation as the population replacement strategy results:\n \
            Average solution value: {result1} \n')
      print(f'Genetic algorithm with acceptNewPopulationIfItHasAtLeastOneFitMember as the population replacement strategy results:\n  \
            Average solution value: {result2} \n')
      print(f'Genetic algorithm with acceptNewPopulationIfItIsBetterThanTheCurrentOne as the population replacement strategy results:\n \
            Average solution value: {result3} \n')
      print(f'Genetic algorithm with acceptNewPopulationIfItIsMuchBetterThanTheCurrentOne as the population replacement strategy results:\n \
            Average solution value: {result4} \n')
    
studyPopulationReplacementStrategies()

# What I observed:
# 1. acceptNewPopulationIfItHasAtLeastOneFitMember works best when genetic_algorithm runs for a smaller NUMBER_OF_GENERATIONS (for example 100)
# 2. acceptNewPopulation, closesly followed by acceptNewPopulationIfItHasAtLeastOneFitMember, is the best when NUMBER_OF_GENERATIONS is larger, (for example 1000)
# This might be because acceptNewPopulation exhausts the search space
# 3. As expected, acceptNewPopulationIfItIsMuchBetterThanTheCurrentOne is the worst one on average, 
# but I believe it sometimes finds much better solutions. I'm too lazy to extend the scope of the assignment again

Genetic algorithm with acceptNewPopulation as the population replacement strategy results:
             Average solution value: 27.21 

Genetic algorithm with acceptNewPopulationIfItHasAtLeastOneFitMember as the population replacement strategy results:
              Average solution value: 27.85 

Genetic algorithm with acceptNewPopulationIfItIsBetterThanTheCurrentOne as the population replacement strategy results:
             Average solution value: 26.95 

Genetic algorithm with acceptNewPopulationIfItIsMuchBetterThanTheCurrentOne as the population replacement strategy results:
             Average solution value: 26.57 

